In [1]:
!pip install pandas numpy scikit-learn streamlit joblib pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Generate synthetic data
np.random.seed(42)

data = pd.DataFrame({
    "Station": np.random.choice(["NDLS", "BCT", "CNB", "HWH", "MAS", "SBC", "PNBE", "JAT", "LKO", "BBS"], 5000),
    "Train_Type": np.random.choice(["Express", "Superfast", "Passenger", "Shatabdi", "Rajdhani"], 5000),
    "Weather": np.random.choice(["Clear", "Rainy", "Foggy", "Stormy", "Cloudy"], 5000),
    "Weekday": np.random.choice(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], 5000),
    "Scheduled_Arrival": np.random.randint(0, 24, 5000) + np.random.random(5000),
    "Delay": np.random.randint(0, 120, 5000)  # Delay in minutes
})

# One-hot encode categorical variables
data = pd.get_dummies(data, columns=["Station", "Train_Type", "Weather", "Weekday"], drop_first=True)

# Split dataset
X = data.drop(columns=["Delay"])
y = data["Delay"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the model
joblib.dump(model, "train_delay_model.pkl")

# Save feature names for later use in the app
joblib.dump(X_train.columns, "feature_names.pkl")

print("✅ Model trained and saved successfully!")


✅ Model trained and saved successfully!


In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import random

# Load the trained model and feature names
model = joblib.load("train_delay_model.pkl")
feature_names = joblib.load("feature_names.pkl")

# Streamlit UI
def main():
    st.title("🚆 Indian Railways Train Delay Predictor")

    # User input selection
    station = st.selectbox("📍 Select Station", ["NDLS", "BCT", "CNB", "HWH", "MAS", "SBC", "PNBE", "JAT", "LKO", "BBS"])
    train_type = st.selectbox("🚆 Select Train Type", ["Express", "Superfast", "Passenger", "Shatabdi", "Rajdhani"])
    weather = st.selectbox("🌤️ Select Weather Condition", ["Clear", "Rainy", "Foggy", "Stormy", "Cloudy"])
    weekday = st.selectbox("📅 Select Day of the Week", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
    arrival_hour = st.slider("⏰ Scheduled Arrival Hour", 0, 23, 10)

    # Convert hour to decimal format
    scheduled_arrival = arrival_hour + random.uniform(0, 0.99)

    # Prediction button
    if st.button("🚦 Predict Delay"):
        # Convert user input to DataFrame
        input_data = pd.DataFrame([[station, train_type, weather, weekday, scheduled_arrival]],
                                  columns=["Station", "Train_Type", "Weather", "Weekday", "Scheduled_Arrival"])

        # One-hot encode categorical variables
        input_encoded = pd.get_dummies(input_data, columns=["Station", "Train_Type", "Weather", "Weekday"], drop_first=True)

        # Ensure input matches model feature shape
        input_encoded = input_encoded.reindex(columns=feature_names, fill_value=0)

        # Make prediction
        prediction = model.predict(input_encoded)[0]

        # Display result
        st.success(f"⏳ Predicted Delay: {prediction:.2f} minutes")

if __name__ == "__main__":
    main()


Writing app.py


In [8]:
!pip install streamlit pyngrok


In [9]:
%%writefile app.py
import streamlit as st

st.title("Train Delay Prediction")
st.write("This is a synthetic data-based AI model for Indian Railways")


Overwriting app.py


In [16]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
changed 22 packages in 2s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [17]:
!streamlit run app.py &>/dev/null &


In [ ]:
!lt --port 8501


your url is: https://new-jars-design.loca.lt
